# Kaggle - WiDS Datathon 2022

## Modelo Nº5

### Description:

- All the features
- Not separation train/test
- Catboost
- Looking hiperparameters by Gridsearch
- Cross validation
- Iterations: 1000

- Public Score = 40.651

## Modelo Nº5A

### Description:

- All the features
- Not separation train/test
- Catboost
- Looking hiperparameters by Gridsearch only in learning rate
- Cross validation
- Iterations: 1000

- Public Score = 48.420


## Modelo Nº5B

### Description:

- All the features
- Not separation train/test
- Catboost
- Looking hiperparameters by Gridsearch only in depth of the trees
- Cross validation
- Iterations: 1000

- Public Score = 37.508


## Modelo Nº5C

### Description:

- All the features
- Not separation train/test
- Catboost
- Learning rate from the model 5B
- Cross validation
- Iterations: 2000
- Early stoping

- Public Score = 35.967

# 1. Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#model
from catboost import CatBoostRegressor

#train/test split
from sklearn.model_selection import train_test_split

#pool class
from catboost import Pool

#cross validation
from catboost import cv

# 2. Data

In [2]:
#paths to the Github repo

train_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/train.csv'
test_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/test.csv'
submit_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/sample_solution.csv'

In [3]:
#train dataset

train = pd.read_csv(train_path)
train.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
#test dataset

test = pd.read_csv(test_path)
test.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75757
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75758
2,7,State_1,Commercial,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75759
3,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75760
4,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,38,50.596774,...,0,29,5,2,0,340.0,330.0,22.8,126.0,75761


In [5]:
#sumbit

submit = pd.read_csv(submit_path)
submit.head()

,id,site_eui
0,75757,0.0
1,75758,0.0
2,75759,0.0
3,75760,0.0
4,75761,0.0


In [6]:
x = train.drop(['id','site_eui'], axis=1)
x.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [7]:
y = train['site_eui']

In [8]:
cat_features = list(train.select_dtypes(include=['object']).columns)
cat_features

['State_Factor', 'building_class', 'facility_type']

### Model 5

In [12]:
model_5 = CatBoostRegressor(cat_features = cat_features)

In [14]:
#grid params 
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

#cross validation proces
grid_search_result = model_5.grid_search(grid, 
                                       X=x, 
                                       y=y, 
                                       #verbose=False,
                                       plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 98.5937442	test: 100.3438567	best: 100.3438567 (0)	total: 29.4ms	remaining: 29.4s
1:	learn: 96.5519938	test: 98.3488199	best: 98.3488199 (1)	total: 49.7ms	remaining: 24.8s
2:	learn: 94.5834507	test: 96.4255100	best: 96.4255100 (2)	total: 69.1ms	remaining: 23s
3:	learn: 92.7080217	test: 94.5959340	best: 94.5959340 (3)	total: 88.7ms	remaining: 22.1s
4:	learn: 90.8807542	test: 92.8124208	best: 92.8124208 (4)	total: 108ms	remaining: 21.4s
5:	learn: 89.1448295	test: 91.1210562	best: 91.1210562 (5)	total: 127ms	remaining: 21.1s
6:	learn: 87.4578517	test: 89.4790947	best: 89.4790947 (6)	total: 148ms	remaining: 20.9s
7:	learn: 85.8700831	test: 87.9350760	best: 87.9350760 (7)	total: 166ms	remaining: 20.6s
8:	learn: 84.3362732	test: 86.4478837	best: 86.4478837 (8)	total: 187ms	remaining: 20.6s
9:	learn: 82.8649363	test: 85.0222038	best: 85.0222038 (9)	total: 206ms	remaining: 20.3s
10:	learn: 81.4477498	test: 83.6497036	best: 83.6497036 (10)	total: 225ms	remaining: 20.3s
11:	learn: 80.0

99:	learn: 47.8792079	test: 50.8585032	best: 50.8585032 (99)	total: 1.8s	remaining: 16.2s
100:	learn: 47.8478405	test: 50.8272198	best: 50.8272198 (100)	total: 1.81s	remaining: 16.1s
101:	learn: 47.7620922	test: 50.7460288	best: 50.7460288 (101)	total: 1.83s	remaining: 16.1s
102:	learn: 47.7318363	test: 50.7144979	best: 50.7144979 (102)	total: 1.85s	remaining: 16.1s
103:	learn: 47.6788912	test: 50.6615528	best: 50.6615528 (103)	total: 1.87s	remaining: 16.1s
104:	learn: 47.6551353	test: 50.6371783	best: 50.6371783 (104)	total: 1.88s	remaining: 16.1s
105:	learn: 47.6250693	test: 50.6051972	best: 50.6051972 (105)	total: 1.9s	remaining: 16s
106:	learn: 47.6073963	test: 50.5875127	best: 50.5875127 (106)	total: 1.92s	remaining: 16s
107:	learn: 47.5802905	test: 50.5613739	best: 50.5613739 (107)	total: 1.93s	remaining: 16s
108:	learn: 47.5542313	test: 50.5389152	best: 50.5389152 (108)	total: 1.95s	remaining: 16s
109:	learn: 47.4703169	test: 50.4617195	best: 50.4617195 (109)	total: 1.97s	remain

189:	learn: 45.7342503	test: 48.8669754	best: 48.8669754 (189)	total: 3.45s	remaining: 14.7s
190:	learn: 45.7107019	test: 48.8491377	best: 48.8491377 (190)	total: 3.48s	remaining: 14.7s
191:	learn: 45.6991049	test: 48.8418410	best: 48.8418410 (191)	total: 3.5s	remaining: 14.7s
192:	learn: 45.6931967	test: 48.8367231	best: 48.8367231 (192)	total: 3.52s	remaining: 14.7s
193:	learn: 45.6615533	test: 48.8089682	best: 48.8089682 (193)	total: 3.54s	remaining: 14.7s
194:	learn: 45.6523044	test: 48.8021053	best: 48.8021053 (194)	total: 3.56s	remaining: 14.7s
195:	learn: 45.6314394	test: 48.7866061	best: 48.7866061 (195)	total: 3.58s	remaining: 14.7s
196:	learn: 45.6053506	test: 48.7655198	best: 48.7655198 (196)	total: 3.6s	remaining: 14.7s
197:	learn: 45.5939687	test: 48.7544004	best: 48.7544004 (197)	total: 3.61s	remaining: 14.6s
198:	learn: 45.5888508	test: 48.7492083	best: 48.7492083 (198)	total: 3.63s	remaining: 14.6s
199:	learn: 45.5636339	test: 48.7266162	best: 48.7266162 (199)	total: 3.

...

575:	learn: 34.4297747	test: 44.3572108	best: 44.3572108 (575)	total: 43.4s	remaining: 31.9s
576:	learn: 34.4121756	test: 44.3494259	best: 44.3494259 (576)	total: 43.5s	remaining: 31.9s
577:	learn: 34.4068399	test: 44.3506784	best: 44.3494259 (576)	total: 43.5s	remaining: 31.8s
578:	learn: 34.3989663	test: 44.3499496	best: 44.3494259 (576)	total: 43.6s	remaining: 31.7s
579:	learn: 34.3855123	test: 44.3502042	best: 44.3494259 (576)	total: 43.7s	remaining: 31.6s
580:	learn: 34.3669459	test: 44.3417825	best: 44.3417825 (580)	total: 43.8s	remaining: 31.6s
581:	learn: 34.3551501	test: 44.3400070	best: 44.3400070 (581)	total: 43.8s	remaining: 31.5s
582:	learn: 34.3297378	test: 44.3399580	best: 44.3399580 (582)	total: 43.9s	remaining: 31.4s
583:	learn: 34.3068681	test: 44.3397112	best: 44.3397112 (583)	total: 44s	remaining: 31.3s
584:	learn: 34.2973489	test: 44.3293359	best: 44.3293359 (584)	total: 44.1s	remaining: 31.3s
585:	learn: 34.2914172	test: 44.3267161	best: 44.3267161 (585)	total: 44

665:	learn: 33.5691067	test: 44.1874539	best: 44.1874539 (665)	total: 50.2s	remaining: 25.2s
666:	learn: 33.5627155	test: 44.1879146	best: 44.1874539 (665)	total: 50.2s	remaining: 25.1s
667:	learn: 33.5566406	test: 44.1879035	best: 44.1874539 (665)	total: 50.3s	remaining: 25s
668:	learn: 33.5433461	test: 44.1854634	best: 44.1854634 (668)	total: 50.4s	remaining: 24.9s
669:	learn: 33.5366262	test: 44.1848168	best: 44.1848168 (669)	total: 50.5s	remaining: 24.9s
670:	learn: 33.5151445	test: 44.1804784	best: 44.1804784 (670)	total: 50.5s	remaining: 24.8s
671:	learn: 33.5107660	test: 44.1794541	best: 44.1794541 (671)	total: 50.6s	remaining: 24.7s
672:	learn: 33.5050634	test: 44.1795927	best: 44.1794541 (671)	total: 50.7s	remaining: 24.6s
673:	learn: 33.4978131	test: 44.1791069	best: 44.1791069 (673)	total: 50.8s	remaining: 24.6s
674:	learn: 33.4936979	test: 44.1777103	best: 44.1777103 (674)	total: 50.8s	remaining: 24.5s
675:	learn: 33.4906379	test: 44.1763252	best: 44.1763252 (675)	total: 50

756:	learn: 32.8145818	test: 44.0997399	best: 44.0997399 (756)	total: 57.1s	remaining: 18.3s
757:	learn: 32.8134183	test: 44.0999364	best: 44.0997399 (756)	total: 57.2s	remaining: 18.3s
758:	learn: 32.8125132	test: 44.1003920	best: 44.0997399 (756)	total: 57.3s	remaining: 18.2s
759:	learn: 32.8118253	test: 44.0989913	best: 44.0989913 (759)	total: 57.4s	remaining: 18.1s
760:	learn: 32.7981111	test: 44.0970257	best: 44.0970257 (760)	total: 57.4s	remaining: 18s
761:	learn: 32.7968503	test: 44.0970732	best: 44.0970257 (760)	total: 57.5s	remaining: 18s
762:	learn: 32.7962228	test: 44.0968394	best: 44.0968394 (762)	total: 57.6s	remaining: 17.9s
763:	learn: 32.7915601	test: 44.0973164	best: 44.0968394 (762)	total: 57.6s	remaining: 17.8s
764:	learn: 32.7904874	test: 44.0980608	best: 44.0968394 (762)	total: 57.7s	remaining: 17.7s
765:	learn: 32.7770753	test: 44.0898200	best: 44.0898200 (765)	total: 57.8s	remaining: 17.7s
766:	learn: 32.7759850	test: 44.0898921	best: 44.0898200 (765)	total: 57.9

845:	learn: 32.2126154	test: 44.0182042	best: 44.0182042 (845)	total: 1m 3s	remaining: 11.6s
846:	learn: 32.2075322	test: 44.0170120	best: 44.0170120 (846)	total: 1m 3s	remaining: 11.5s
847:	learn: 32.1997128	test: 44.0149261	best: 44.0149261 (847)	total: 1m 3s	remaining: 11.5s
848:	learn: 32.1972803	test: 44.0146858	best: 44.0146858 (848)	total: 1m 4s	remaining: 11.4s
849:	learn: 32.1893634	test: 44.0164014	best: 44.0146858 (848)	total: 1m 4s	remaining: 11.3s
850:	learn: 32.1829816	test: 44.0162877	best: 44.0146858 (848)	total: 1m 4s	remaining: 11.2s
851:	learn: 32.1671701	test: 44.0109756	best: 44.0109756 (851)	total: 1m 4s	remaining: 11.2s
852:	learn: 32.1597318	test: 44.0110279	best: 44.0109756 (851)	total: 1m 4s	remaining: 11.1s
853:	learn: 32.1509243	test: 44.0128668	best: 44.0109756 (851)	total: 1m 4s	remaining: 11s
854:	learn: 32.1431735	test: 44.0095436	best: 44.0095436 (854)	total: 1m 4s	remaining: 10.9s
855:	learn: 32.1314695	test: 44.0100423	best: 44.0095436 (854)	total: 1m

935:	learn: 31.5303640	test: 43.9841574	best: 43.9839150 (933)	total: 1m 10s	remaining: 4.82s
936:	learn: 31.5277069	test: 43.9850658	best: 43.9839150 (933)	total: 1m 10s	remaining: 4.75s
937:	learn: 31.5211124	test: 43.9830950	best: 43.9830950 (937)	total: 1m 10s	remaining: 4.67s
938:	learn: 31.5198838	test: 43.9830872	best: 43.9830872 (938)	total: 1m 10s	remaining: 4.6s
939:	learn: 31.5097576	test: 43.9836598	best: 43.9830872 (938)	total: 1m 10s	remaining: 4.52s
940:	learn: 31.5089011	test: 43.9837478	best: 43.9830872 (938)	total: 1m 10s	remaining: 4.45s
941:	learn: 31.5078720	test: 43.9840023	best: 43.9830872 (938)	total: 1m 11s	remaining: 4.37s
942:	learn: 31.5056759	test: 43.9837983	best: 43.9830872 (938)	total: 1m 11s	remaining: 4.3s
943:	learn: 31.5039934	test: 43.9840856	best: 43.9830872 (938)	total: 1m 11s	remaining: 4.22s
944:	learn: 31.5025054	test: 43.9840227	best: 43.9830872 (938)	total: 1m 11s	remaining: 4.14s
945:	learn: 31.4835894	test: 43.9843447	best: 43.9830872 (938)

In [20]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 7, 'learning_rate': 0.1}

In [9]:
x_test = test.drop('id', axis=1)
x_test.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [17]:
submit['site_eui'] = model_5.predict(x_test)
submit.to_csv('modelo_5.csv',index=False)

## Model 5A

In [16]:
model_5a = CatBoostRegressor(cat_features = cat_features,verbose=False)

In [17]:
#grid params 
grid_5a = {'learning_rate': np.linspace(0.01,0.5,20)}

#cross validation proces
grid_search_result_5a = model_5a.grid_search(grid_5a, 
                                       X=x, 
                                       y=y,
                                       plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 47.27720793
bestIteration = 999

0:	loss: 47.2772079	best: 47.2772079 (0)	total: 28.2s	remaining: 8m 56s

bestTest = 46.08959324
bestIteration = 999

1:	loss: 46.0895932	best: 46.0895932 (1)	total: 56.8s	remaining: 8m 31s

bestTest = 45.3150649
bestIteration = 998

2:	loss: 45.3150649	best: 45.3150649 (2)	total: 1m 25s	remaining: 8m 7s

bestTest = 45.10327046
bestIteration = 997

3:	loss: 45.1032705	best: 45.1032705 (3)	total: 1m 55s	remaining: 7m 42s

bestTest = 44.59296057
bestIteration = 997

4:	loss: 44.5929606	best: 44.5929606 (4)	total: 2m 24s	remaining: 7m 14s

bestTest = 44.01114147
bestIteration = 998

5:	loss: 44.0111415	best: 44.0111415 (5)	total: 2m 55s	remaining: 6m 49s

bestTest = 44.35630066
bestIteration = 998

6:	loss: 44.3563007	best: 44.0111415 (5)	total: 3m 25s	remaining: 6m 20s

bestTest = 44.01763669
bestIteration = 990

7:	loss: 44.0176367	best: 44.0111415 (5)	total: 3m 54s	remaining: 5m 52s

bestTest = 43.80641092
bestIteration = 983

8:	loss: 43.806

In [18]:
submit['site_eui'] = model_5a.predict(x_test)
submit.to_csv('modelo_5a.csv',index=False)

## Model 5B

In [28]:
model_5b = CatBoostRegressor(cat_features = cat_features,verbose=False)

In [29]:
#grid params 
grid_5b = {'depth': np.arange(1,15)}

#cross validation proces
grid_search_result_5b = model_5b.grid_search(grid_5b, 
                                       X=x, 
                                       y=y, 
                                       plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 50.14525237
bestIteration = 999

0:	loss: 50.1452524	best: 50.1452524 (0)	total: 6.33s	remaining: 1m 22s

bestTest = 48.29207275
bestIteration = 998

1:	loss: 48.2920728	best: 48.2920728 (1)	total: 16.9s	remaining: 1m 41s

bestTest = 47.59267492
bestIteration = 998

2:	loss: 47.5926749	best: 47.5926749 (2)	total: 31.6s	remaining: 1m 55s

bestTest = 47.05193298
bestIteration = 998

3:	loss: 47.0519330	best: 47.0519330 (3)	total: 49.9s	remaining: 2m 4s

bestTest = 46.70639936
bestIteration = 999

4:	loss: 46.7063994	best: 46.7063994 (4)	total: 1m 12s	remaining: 2m 10s

bestTest = 46.09515037
bestIteration = 995

5:	loss: 46.0951504	best: 46.0951504 (5)	total: 1m 40s	remaining: 2m 14s

bestTest = 45.88739356
bestIteration = 998

6:	loss: 45.8873936	best: 45.8873936 (6)	total: 2m 14s	remaining: 2m 14s

bestTest = 45.59554348
bestIteration = 999

7:	loss: 45.5955435	best: 45.5955435 (7)	total: 2m 56s	remaining: 2m 12s

bestTest = 45.17249267
bestIteration = 997

8:	loss: 45.1724

In [30]:
submit['site_eui'] = model_5b.predict(x_test)
submit.to_csv('modelo_5b.csv',index=False)

## Modelo 5C

In [35]:
model_5b.learning_rate_

0.08111999928951263

In [10]:
model_5c = model_5c = CatBoostRegressor(cat_features=cat_features,
    iterations=2000,
    learning_rate=0.08111999928951263,
    early_stopping_rounds=50,
    verbose=50
)

In [11]:
#grid params 
grid_5c = {'depth': [13]}

#cross validation proces
grid_search_result_5c = model_5c.grid_search(grid_5c, 
                                       X=x, 
                                       y=y, 
                                       cv=5,
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 94.9562698	test: 96.7869052	best: 96.7869052 (0)	total: 591ms	remaining: 19m 42s
50:	learn: 41.2516208	test: 47.2778924	best: 47.2778924 (50)	total: 20.9s	remaining: 13m 20s
100:	learn: 38.2987250	test: 46.5422927	best: 46.5422927 (100)	total: 40.4s	remaining: 12m 38s
150:	learn: 36.2911386	test: 46.2888688	best: 46.2863923 (149)	total: 59.4s	remaining: 12m 7s
200:	learn: 34.6751458	test: 45.9621564	best: 45.9621564 (200)	total: 1m 19s	remaining: 11m 49s
250:	learn: 33.4001811	test: 45.7453849	best: 45.7441540 (249)	total: 1m 39s	remaining: 11m 33s
300:	learn: 32.2701432	test: 45.5561624	best: 45.5554076 (295)	total: 2m	remaining: 11m 17s
350:	learn: 31.2713156	test: 45.4557049	best: 45.4557025 (349)	total: 2m 20s	remaining: 10m 59s
400:	learn: 30.3652542	test: 45.3429383	best: 45.3375690 (399)	total: 2m 40s	remaining: 10m 39s
450:	learn: 29.4523120	test: 45.2523725	best: 45.2523725 (450)	total: 3m 1s	remaining: 10m 22s
500:	learn: 28.7299903	test: 45.1911703	best: 45.1847330

850:	learn: 25.5300745	test: 40.5417046	best: 40.5408605 (849)	total: 5m 22s	remaining: 7m 15s
900:	learn: 25.0375101	test: 40.4795068	best: 40.4795068 (900)	total: 5m 42s	remaining: 6m 57s
950:	learn: 24.5990492	test: 40.4225926	best: 40.4225926 (950)	total: 6m 3s	remaining: 6m 40s
1000:	learn: 24.2301816	test: 40.3660841	best: 40.3658930 (999)	total: 6m 22s	remaining: 6m 22s
1050:	learn: 23.8524019	test: 40.3207192	best: 40.3207192 (1050)	total: 6m 42s	remaining: 6m 3s
1100:	learn: 23.4646866	test: 40.3036370	best: 40.3011914 (1095)	total: 7m 2s	remaining: 5m 44s
1150:	learn: 23.1268714	test: 40.2609523	best: 40.2588040 (1148)	total: 7m 22s	remaining: 5m 26s
1200:	learn: 22.7817336	test: 40.2337492	best: 40.2336573 (1199)	total: 7m 42s	remaining: 5m 7s
1250:	learn: 22.4494510	test: 40.2025102	best: 40.2025102 (1250)	total: 8m 3s	remaining: 4m 49s
1300:	learn: 22.1145595	test: 40.1849652	best: 40.1842629 (1296)	total: 8m 23s	remaining: 4m 30s
1350:	learn: 21.8184194	test: 40.1623485	b

In [12]:
submit['site_eui'] = model_5c.predict(x_test)
submit.to_csv('modelo_5c.csv',index=False)